In [8]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

Imported player data and created table of usefull bits.

In [9]:
master = pd.read_csv('data/core/Master.csv', dtype=str)
master = master[['playerID', 'nameGiven', 'deathYear', 'finalGame']]
master.head()

,playerID,nameGiven,deathYear,finalGame
0,aardsda01,David Allan,NaN,2015-08-23
1,aaronha01,Henry Louis,NaN,1976-10-03
2,aaronto01,Tommie Lee,1984,1971-09-26
3,aasedo01,Donald William,NaN,1990-10-03
4,abadan01,Fausto Andres,NaN,2006-04-13


Deleted all players known to be dead from the list.

In [10]:
master = master[pd.isnull(master.deathYear)]
master = master.drop('deathYear', 1)
master.head()

,playerID,nameGiven,finalGame
0,aardsda01,David Allan,2015-08-23
1,aaronha01,Henry Louis,1976-10-03
3,aasedo01,Donald William,1990-10-03
4,abadan01,Fausto Andres,2006-04-13
5,abadfe01,Fernando Antonio,2015-10-03


There is an abberation suggesting that 1300 people had their final game in 2015. We can assume that they did not all retire, since it's about 200 each year before that. I created a new variable to hold all players that retired before 2015 and deleted everyone with a value there. This allowed me to keep the players who had a NaN value for final game, as well as those who supposedly retired in 2015.

In [11]:
# master.finalGame = pd.to_datetime(master.finalGame, format='%Y-%m-%d')
master['is_inactive'] = master.finalGame[master.finalGame < '2015-01-01']
master = master[pd.notnull(master.is_inactive)]
master = master.drop(['is_inactive', 'finalGame'], 1)
master.head()

,playerID,nameGiven
1,aaronha01,Henry Louis
3,aasedo01,Donald William
4,abadan01,Fausto Andres
12,abbotgl01,William Glenn
13,abbotje01,Jeffrey William


Filtered player data by players without a final game (players that have not yet retired) and got rid of the column. Now I have a list of player ids and names for active players. There are 8114 living players in this database that had not retired as of 2014.

In [12]:
batting = pd.read_csv('data/core/Batting.csv')
batting = batting[['playerID', 'G', 'AB', 'H', 'BB', 'HBP', 'SF']]
batting.head()

,playerID,G,AB,H,BB,HBP,SF
0,abercda01,1,4.0,0.0,0.0,NaN,NaN
1,addybo01,25,118.0,32.0,4.0,NaN,NaN
2,allisar01,29,137.0,40.0,2.0,NaN,NaN
3,allisdo01,27,133.0,44.0,0.0,NaN,NaN
4,ansonca01,25,120.0,39.0,2.0,NaN,NaN


In [13]:
batting_p = pd.merge(batting, master)
batting_p.head()
batting_p.describe()

,G,AB,H,BB,HBP,SF
count,50484.000000,45635.000000,45635.000000,45635.000000,45441.000000,44822.000000
mean,51.294826,141.345875,36.581944,13.601863,1.048855,1.173553
std,46.614485,184.533605,51.667585,21.128005,2.245112,2.041726
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,4.000000,0.000000,0.000000,0.000000,0.000000
50%,34.000000,48.000000,8.000000,3.000000,0.000000,0.000000
75%,78.000000,229.500000,57.000000,20.000000,1.000000,2.000000
max,165.000000,705.000000,240.000000,232.000000,50.000000,18.000000


In [14]:
batting_p = batting_p.groupby(['playerID']).mean()
batting_p.head()

,G,AB,H,BB,HBP,SF
playerID,,,,,,
aaronha01,143.391304,537.565217,163.956522,60.956522,1.391304,5.26087
aasedo01,34.461538,2.500000,0.000000,0.000000,0.000000,0.00000
abadan01,5.000000,7.000000,0.666667,1.333333,0.000000,0.00000
abbotgl01,20.666667,0.000000,0.000000,0.000000,0.000000,0.00000
abbotje01,46.600000,119.200000,31.400000,7.600000,0.600000,1.40000


In [17]:
batting_p['OBP'] = ((batting_p.H + batting_p.BB + batting_p.HBP)/(batting_p.AB + batting_p.BB + batting_p.HBP + batting_p.SF))

In [18]:
batting_p.head()

,G,AB,H,BB,HBP,SF,OBP
playerID,,,,,,,
aaronha01,143.391304,537.565217,163.956522,60.956522,1.391304,5.26087,0.373949
aasedo01,34.461538,2.500000,0.000000,0.000000,0.000000,0.00000,0.000000
abadan01,5.000000,7.000000,0.666667,1.333333,0.000000,0.00000,0.240000
abbotgl01,20.666667,0.000000,0.000000,0.000000,0.000000,0.00000,NaN
abbotje01,46.600000,119.200000,31.400000,7.600000,0.600000,1.40000,0.307453
